# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [6]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import pandas as pd

In [7]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [23]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
4282,Color,Kenneth Branagh,85.0,150.0,0.0,401.0,Brian Blessed,597.0,4414535.0,Drama,...,224.0,English,UK,PG-13,18000000.0,1996.0,591.0,7.8,2.20,0
759,Color,Geoff Murphy,47.0,92.0,14.0,488.0,Brenda Bakke,789.0,50024083.0,Action|Thriller,...,126.0,English,USA,R,60000000.0,1995.0,520.0,5.4,1.85,1000
3192,Color,Steven Soderbergh,111.0,89.0,0.0,132.0,Lesley Ann Warren,402.0,3193102.0,Crime|Drama|Mystery|Thriller,...,240.0,English,USA,R,9000000.0,1999.0,296.0,7.1,1.85,944
293,Color,Paul Weitz,180.0,98.0,80.0,708.0,Blythe Danner,22000.0,148383780.0,Comedy|Romance,...,152.0,English,USA,PG-13,100000000.0,2010.0,713.0,5.5,1.85,0
1891,Color,Ivan Reitman,245.0,108.0,425.0,766.0,Greta Gerwig,20000.0,70625986.0,Comedy|Romance,...,178.0,English,USA,R,25000000.0,2011.0,962.0,6.2,2.35,24000


In [24]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.0013880626611144642
                  actor_1_name: 0.0013880626611144642
        actor_2_facebook_likes: 0.0025778306563553066
                  actor_2_name: 0.0025778306563553066
        actor_3_facebook_likes: 0.004560777315090192
                  actor_3_name: 0.004560777315090192
                  aspect_ratio: 0.0652389450723776
                        budget: 0.09756097560975607
     cast_total_facebook_likes: 0.0
                         color: 0.00376759865159626
                content_rating: 0.06008328375966687
                       country: 0.0009914733293674427
       director_facebook_likes: 0.020622645250842786
                 director_name: 0.020622645250842786
                      duration: 0.002974419988102328
          facenumber_in_poster: 0.0025778306563553066
                        genres: 0.0
                         gross: 0.1752924846321634
                    imdb_score: 0.0
                      language: 0.00237953599